In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torchvision as tv
from torchsummary import summary

## Данные

In [2]:
BATCH_SIZE=256

In [3]:
train_dataset = tv.datasets.FashionMNIST('.', train=True, transform=tv.transforms.ToTensor(), download=True)
test_dataset = tv.datasets.FashionMNIST('.', train=False, transform=tv.transforms.ToTensor(), download=True)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

## Функция для обучения

In [4]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = torch.Tensor([0]), 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

def train(net, train_iter, test_iter, optimizer, num_epochs):
    loss = nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        
        for X, y in train_iter:
            optimizer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
        
        test_acc = evaluate_accuracy(test_iter, net)
        print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}' \
              f', test acc {test_acc:.3f}, time {time.time() - start:.1f} sec')

## Модель

In [5]:
model = nn.Sequential(
    nn.Conv2d(1, 8, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.AvgPool2d(2, stride=2),
    nn.Conv2d(8, 16, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.AvgPool2d(2, stride=2),
    nn.Conv2d(16, 32, kernel_size=3, padding=1),
    nn.Flatten(),
    nn.Linear(1568, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

summary(model, input_size=(1, 28, 28), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
              ReLU-2            [-1, 8, 28, 28]               0
         AvgPool2d-3            [-1, 8, 14, 14]               0
            Conv2d-4           [-1, 16, 14, 14]           1,168
              ReLU-5           [-1, 16, 14, 14]               0
         AvgPool2d-6             [-1, 16, 7, 7]               0
            Conv2d-7             [-1, 32, 7, 7]           4,640
           Flatten-8                 [-1, 1568]               0
            Linear-9                  [-1, 128]         200,832
             ReLU-10                  [-1, 128]               0
           Linear-11                   [-1, 10]           1,290
Total params: 208,010
Trainable params: 208,010
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/

In [6]:
optimizer = torch.optim.Adam(model.parameters())
train(model, train_iter, test_iter, optimizer, num_epochs=30)

epoch 1, loss 0.0031, train acc 0.716, test acc 0.802, time 10.5 sec
epoch 2, loss 0.0018, train acc 0.833, test acc 0.840, time 10.2 sec
epoch 3, loss 0.0016, train acc 0.856, test acc 0.849, time 11.6 sec
epoch 4, loss 0.0014, train acc 0.868, test acc 0.859, time 11.7 sec
epoch 5, loss 0.0013, train acc 0.875, test acc 0.860, time 9.9 sec
epoch 6, loss 0.0013, train acc 0.882, test acc 0.870, time 10.2 sec
epoch 7, loss 0.0012, train acc 0.887, test acc 0.876, time 11.9 sec
epoch 8, loss 0.0012, train acc 0.890, test acc 0.878, time 10.1 sec
epoch 9, loss 0.0011, train acc 0.895, test acc 0.881, time 9.9 sec
epoch 10, loss 0.0011, train acc 0.899, test acc 0.885, time 10.1 sec
epoch 11, loss 0.0010, train acc 0.902, test acc 0.888, time 12.2 sec
epoch 12, loss 0.0010, train acc 0.905, test acc 0.889, time 10.6 sec
epoch 13, loss 0.0010, train acc 0.908, test acc 0.891, time 10.1 sec
epoch 14, loss 0.0009, train acc 0.911, test acc 0.891, time 11.1 sec
epoch 15, loss 0.0009, train ac